In [ ]:
import os
import pandas as pd
import helpers.spotify_web_api as sp
print(sp.get_track_features('1aKXOi3zQJ6VOdhnkFJ2gZ'))

DIR_DATA = os.path.join('..', 'data')
DIR_PROCESSED = os.path.join(DIR_DATA, 'processed')
DIR_RAW = os.path.join(DIR_DATA, 'raw')

FILENAME_OUT = 'track_audio_features.csv'
PATH_OUT = os.path.join(DIR_RAW, FILENAME_OUT)

FILENAME_TRACKLIST = 'unique_track_ids.pkl'
PATH_TRACKLIST = os.path.join(DIR_PROCESSED, FILENAME_TRACKLIST)

# TODO: colnames

In [ ]:
#TODO: read existing file

In [ ]:
#TODO: create new dataframe

In [ ]:
#TODO: logic for adding data frames together